# * Import Vinsight Data

In [5]:
import os
import glob
import configparser
import oracledb
import pandas as pd
import datetime as dt

## Parameter

In [6]:
config = configparser.ConfigParser()
config.read('../my_config.ini')
config.sections()

username = config['TDMDBPR']['username']
password = config['TDMDBPR']['password']
db = config['TDMDBPR']['db']
host = config['TDMDBPR']['host']
port = config['TDMDBPR']['port']

In [7]:
raw_dir = f"../data/raw"
curr_dt = dt.datetime.now().date()
str_curr_dt = curr_dt.strftime('%Y%m%d')
op_file_name = f'Agg_Perf_Nationwide-{str_curr_dt}'
op_raw_file = f'{raw_dir}/{op_file_name}.csv'

op_raw_file

'../data/raw/Agg_Perf_Nationwide-20240605.csv'

## Query

In [34]:
# TDMDBPR

# conn = oracledb.connect(user=username, password=password, dsn=f'{host}/{db}')
dsn = f'{username}/{password}@{host}:{port}/{db}'
conn = oracledb.connect(dsn)
print(f'{db} : Connected')

cur = conn.cursor()

v_date = (curr_dt, )

# query = """
#     SELECT TM_KEY_DAY, TM_KEY_WK, TM_KEY_MTH, TM_KEY_QTR, TM_KEY_YR, CENTER, PRODUCT_GRP, COMP_CD, METRIC_GRP, METRIC_NAME_GROUP, METRIC_CD, METRIC_NAME, AREA_TYPE, AREA_CD, AREA_NAME
#         , ACTUAL_AS_OF, AGG_TYPE, RR_IND, GRY_IND, UOM, PERIOD, ACTUAL_SNAP, ACTUAL_AGG, TARGET_SNAP, TARGET_AGG, BASELINE_SNAP, BASELINE_AGG, ACH_SNAP, ACH_AGG, GAP_SNAP, GAP_AGG
#         , WOW, WOW_PERCENT, MOM, MOM_PERCENT, QOQ, QOQ_PERCENT, YOY, YOY_PERCENT, RR, RR_ACH, WTD, MTD, QTD, YTD, PPN_TM
#     FROM GEOSPCAPPO.AGG_PERF_NEWCO NOLOCK
#     WHERE CENTER IN ('Revenue', 'Sales')
#     AND AREA_TYPE = 'P'
# """

with open('SQL/Import Vinsight Data.sql', 'r') as sql_file:
    queries = sql_file.read().split(';')
    query = queries[0].strip()
    sql_file.close()
    # print(query)

try:
    print(f'\nExecute query as of {v_date[0]}...')
    cur.execute(query)
    rows = cur.fetchall()
    df = pd.DataFrame.from_records(rows, columns=[x[0] for x in cur.description])
    print(f'\ndf : {df.shape[0]} rows, {df.shape[1]} columns\nDone !!!')


except oracledb.DatabaseError as e:
    print(f'Error with Oracle : {e}')


finally:
    cur.close()
    conn.close()
    print(f'\n{db} : Disconnected')

TDMDBPR : Connected
SELECT TM_KEY_DAY, TM_KEY_WK, TM_KEY_MTH, TM_KEY_QTR, TM_KEY_YR, CENTER, PRODUCT_GRP, COMP_CD, METRIC_GRP, METRIC_NAME_GROUP, METRIC_CD, METRIC_NAME, AREA_TYPE, AREA_CD, AREA_NAME
    , ACTUAL_AS_OF, AGG_TYPE, RR_IND, GRY_IND, UOM, PERIOD, ACTUAL_SNAP, ACTUAL_AGG, TARGET_SNAP, TARGET_AGG, BASELINE_SNAP, BASELINE_AGG, ACH_SNAP, ACH_AGG, GAP_SNAP, GAP_AGG
    , WOW, WOW_PERCENT, MOM, MOM_PERCENT, QOQ, QOQ_PERCENT, YOY, YOY_PERCENT, RR, RR_ACH, WTD, MTD, QTD, YTD, PPN_TM
FROM GEOSPCAPPO.AGG_PERF_NEWCO
WHERE CENTER IN ('Revenue', 'Sales')
AND AREA_TYPE = 'P'


## Sample data

In [5]:
# df.columns
# df.dtypes
# df.describe()
df.head()

,TM_KEY_DAY,TM_KEY_WK,TM_KEY_MTH,TM_KEY_QTR,TM_KEY_YR,CENTER,PRODUCT_GRP,COMP_CD,METRIC_GRP,METRIC_NAME_GROUP,...,QOQ_PERCENT,YOY,YOY_PERCENT,RR,RR_ACH,WTD,MTD,QTD,YTD,PPN_TM
0,20230301,2023009,202303,20231,2023,Revenue,Postpaid,DTAC,Revenue,Postpaid Revenue,...,NaN,None,None,4.266777e+08,14.079643,None,None,None,None,2024-04-23 12:22:16
1,20230301,2023009,202303,20231,2023,Revenue,Prepaid,TRUE,Sales,Prepaid Number of AP 1D,...,NaN,None,None,7.994900e+04,NaN,None,None,None,None,2024-04-23 12:22:16
2,20230301,2023009,202303,20231,2023,Revenue,Prepaid,TRUE,Sales,Postpaid GA RC,...,NaN,None,None,NaN,NaN,None,None,None,None,2024-04-23 12:22:16
3,20230301,2023009,202303,20231,2023,Revenue,Postpaid,TRUE,Sales,Postpaid Gross Adds,...,NaN,None,None,NaN,NaN,None,None,None,None,2024-04-23 12:22:16
4,20230301,2023009,202303,20231,2023,Revenue,Postpaid,ALL,Sales,With Device (Excld MNP) B2C,...,NaN,None,None,7.086600e+04,NaN,None,None,None,None,2024-04-23 12:22:16


## Generate Output file

In [6]:
df.to_csv(f'{op_raw_file}', index=False, encoding='utf-8')

print(f'\n"{op_raw_file}" is generated')


"../data/raw/Agg_Perf_Nationwide-20240605.csv" is generated


In [7]:
os.chdir('../data/raw')
# cmd_cwd = os.getcwd()
# cmd_cwd

# cmd_rm = f'rm *.tar.gz'
# os.system(cmd_rm)
# cmd_rm

# Remove old files
old_files = glob.glob('Agg_Perf_Nationwide*.tar.gz')
for file in old_files:
    os.remove(file)

# Add tar.gz
cmd_tar = f'Tar -cvzf {op_file_name}.tar.gz {op_file_name}.csv'
os.system(cmd_tar)

0

### Manual Step

In [8]:
op_file_name

# cd data/raw
# Tar -cvzf Agg_Perf_Nationwide-20240604.tar.gz Agg_Perf_Nationwide-20240604.csv

'Agg_Perf_Nationwide-20240605'